In [1]:
# If You use in Colab, You Should run this script
import os
if (not os.path.exists("./SSD-object-detection") and
    not "SSD-object-detection" in os.getcwd()):
    !git clone https://github.com/public-ai/SSD-object-detection.git
    os.chdir("./SSD-object-detection")

In [2]:
%matplotlib inline

import sys
sys.path.append("../")
from utils import DetectionDataset, draw_rectangle
from models.generator import DetectionGenerator, PriorBoxes
from models.ssd import build_base_network, attach_multibox_head, remodel_fpn_network
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

np.random.seed(10)
tf.random.set_random_seed(10)

# \[ 모델 학습 \]
----
----


In [3]:
strides = [4, 8, 16]
scales = [10, 25, 40]
ratios = [(1,1),     # ratio : 1.
          (0.5,1.5), # ratio : 0.33
          (0.8,1.2), # ratio : 0.67
          (1.2,0.8), # ratio : 1.5
          (1.4,1.4)]

prior = PriorBoxes(strides,scales,ratios)

In [6]:
base_network = build_base_network(num_units=16)
fpn_network = remodel_fpn_network(base_network, ['norm3_2','norm4_2','norm5_2'])
predictions = attach_multibox_head(fpn_network,
                                   ['P1','P2','P3'],
                                   num_priors=len(ratios))
model = Model(base_network.input,
              predictions,
              name='ssd')

## 2. 데이터 구성하기
---


In [7]:
trainset = DetectionDataset(data_type='train')
validset = DetectionDataset(data_type='validation')

# .config로 했을 때에만 use_multiprocessing이 가능함
# Argument로 들어가는 것들이 Serialize되는 Class이어야 되기 때문
traingen = DetectionGenerator(trainset.config, 
                              prior.config, 
                              best_match_policy=True,
                              batch_size=64)
validgen = DetectionGenerator(validset.config, 
                              prior.config, 
                              best_match_policy=True,                              
                              batch_size=64)

## 4. 모델 컴파일 및 학습하기
----


In [8]:
from models.losses import SSDLoss
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [9]:
model.compile(Adam(1e-3),
              loss=SSDLoss(1.0,3.,ignore_match=True))

callbacks =[]
rlrop = ReduceLROnPlateau(factor=0.1, 
                          min_lr=1e-6, 
                          patience=5,
                          cooldown=3)
callbacks.append(rlrop)

W0730 14:57:44.309578 4470715840 deprecation.py:323] From ../models/losses.py:37: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0730 14:57:44.326915 4470715840 deprecation_wrapper.py:119] From ../models/losses.py:48: The name tf.log is deprecated. Please use tf.math.log instead.



In [ ]:
model.fit_generator(traingen,
                    epochs=50,
                    validation_data=validgen,
                    use_multiprocessing=True,
                    workers=6,
                    callbacks=callbacks)

## 5. 모델을 통해 예측하기
----

모델을 학습시키는 데에 시간이 걸리므로, 우선 학습된 모델을 이용해보도록 하겠습니다.